In [1]:
import argparse
import sys
from torchvision.datasets import MNIST
from mynn.data import mnist_augment, preprocess, basic_mnist_augment, merge_datasets
from mynn.layer import Flatten, Linear, ReLU, He, Conv, Dropout, Pooling, BN,Sigmoid
from mynn.loss import CrossEntropy
from mynn import Model
from mynn.optimizer import SGD, Adam, MomentGD
from mynn.runner import RunnerM, EarlyStopping, CosineAnnealingLR
import cupy as cp
from mynn.attack import *

# basic test

In [2]:
cp.random.seed(42)

train_dataset = MNIST(
    root="dataset",
    train=True,
    transform=basic_mnist_augment(train=True),
    download=False
)
test_dataset = MNIST(
    root="dataset",
    train=False,
    transform=basic_mnist_augment(train=False),
    download=False
)


train_images, train_labels = preprocess(train_dataset)
test_images, test_labels = preprocess(test_dataset)



train_set = (train_images[:50000], train_labels[:50000])

dev_set = (train_images[50000:], train_labels[50000:])
test_images = cp.asarray(test_images)
test_labels = cp.asarray(test_labels)
test_set = (test_images, test_labels)

In [3]:
model = Model().load_model('best_model_basic/minst_base_lenet/best_model.pickle')
loss_fn   = CrossEntropy(model=model)
runner    = RunnerM(model=model, loss=loss_fn, optimizer=None)

Model loaded from: best_model_basic\minst_base_lenet\best_model.pickle


In [4]:
# 4. 测试集评估
test_loss, test_acc = runner.evaluate(test_set, batch_size=128,loss_fn=loss_fn)
print(f"Test loss: {test_loss:.4f}, Test accuracy: {test_acc:.4f}")

Test loss: 0.0706, Test accuracy: 0.9776


In [7]:
from mynn.attack import fgsm_attack
adv_test_images = fgsm_attack(
    model=model, 
    images=test_images, 
    labels=test_labels, 
    loss_fn=loss_fn,
    epsilon=20/255.0  
)

test_set_adv = (adv_test_images, test_labels)

In [9]:
adv_loss, adv_acc = runner.evaluate(test_set_adv, batch_size=128, loss_fn=loss_fn)
print(f"Adversarial Test loss: {adv_loss:.5f}, Test accuracy: {adv_acc:.5f}")

Adversarial Test loss: 3.44903, Test accuracy: 0.25574


In [10]:
from mynn.attack import fgsm_attack
adv_test_images = fgsm_attack(
    model=model, 
    images=test_images, 
    labels=test_labels, 
    loss_fn=loss_fn,
    epsilon=5/255.0  
)

test_set_adv = (adv_test_images, test_labels)

In [11]:
adv_loss, adv_acc = runner.evaluate(test_set_adv, batch_size=128, loss_fn=loss_fn)
print(f"Adversarial Test loss: {adv_loss:.5f}, Test accuracy: {adv_acc:.5f}")

Adversarial Test loss: 0.25628, Test accuracy: 0.91851


In [12]:
model   = Model().load_model('best_models_fgsm/minst-bn-fgsm/best_model.pickle')
loss_fn   = CrossEntropy(model=model)
runner    = RunnerM(model=model, loss=loss_fn, optimizer=None)

Model loaded from: best_models_fgsm\minst-bn-fgsm\best_model.pickle


In [13]:
# 4. 测试集评估
test_loss, test_acc = runner.evaluate(test_set, batch_size=128,loss_fn=loss_fn)
print(f"Test loss: {test_loss:.4f}, Test accuracy: {test_acc:.4f}")

Test loss: 0.0336, Test accuracy: 0.9889


In [14]:
from mynn.attack import fgsm_attack
adv_test_images = fgsm_attack(
    model=model, 
    images=test_images, 
    labels=test_labels, 
    loss_fn=loss_fn,
    epsilon=20/255.0  
)

test_set_adv = (adv_test_images, test_labels)

In [15]:
adv_loss, adv_acc = runner.evaluate(test_set_adv, batch_size=128, loss_fn=loss_fn)
print(f"Adversarial Test loss: {adv_loss:.5f}, Test accuracy: {adv_acc:.5f}")

Adversarial Test loss: 0.11101, Test accuracy: 0.96292


In [16]:
from mynn.attack import fgsm_attack

adv_test_images = fgsm_attack(
    model=model,
    images=test_images,
    labels=test_labels,
    loss_fn=loss_fn,
    epsilon=5 / 255.0
)

test_set_adv = (adv_test_images, test_labels)
adv_loss, adv_acc = runner.evaluate(test_set_adv, batch_size=128, loss_fn=loss_fn)
print(f"Adversarial Test loss: {adv_loss:.5f}, Test accuracy: {adv_acc:.5f}")

Adversarial Test loss: 0.04838, Test accuracy: 0.98418
